In [ ]:
import sqlite3
import pandas as pd
from typing import Dict, Any

conn = sqlite3.connect("test_database_")
cursor = conn.cursor()

def convert_csv_into_table(
    connection: sqlite3.Connection,
    cursor: sqlite3.Connection.cursor,
    csv_filename: str,
    table_name: str,
    columns_dict: Dict[str, str],
    read_csv_args: Dict[str, Any] = {}
):
  dataframe = pd.read_csv(csv_filename, **read_csv_args)

  insert_execute_string = f'INSERT INTO {table_name} ({{}}) VALUES ({{}});'

  for record in dataframe.to_dict('records'):

    record = {columns_dict[key]: value for key, value in record.items() if not pd.isnull(value)}
    values_string = ','.join([
      str(i) if type(i) != str else f'"{i}"' for i in record.values() 
    ])
    
    execute_string = insert_execute_string.format(
        ','.join(list(record.keys())),
        values_string
    )
    cursor.execute(execute_string)
  conn.commit()


In [ ]:
cursor.execute(
    """
    CREATE TABLE hist_painel_covidbr_2020_parte1nov2021 (
      id INTEGER PRIMARY KEY AUTOINCREMENT,
      regiao TEXT,
      estado TEXT,
      municipio TEXT,
      coduf INTEGER,
      codmun REAL,
      codRegiaoSaude REAL,
      nomeRegiaoSaude TEXT,
      data TEXT,
      semanaEpi INTEGER,
      populacaoTCU2019 REAL,
      casosAcumulado INTEGER,
      casosNovos INTEGER,
      obitosAcumulado INTEGER,
      obitosNovos INTEGER,
      recuperadosNovos REAL,
      emAcompanhamentoNovos REAL,
      interior_metropolitana REAL
    );"""
)

In [ ]:
for path in [
  '/content/HIST_PAINEL_COVIDBR_2020_Parte1_10nov2021.csv',
  '/content/HIST_PAINEL_COVIDBR_2020_Parte2_10nov2021.csv',
  '/content/HIST_PAINEL_COVIDBR_2021_Parte1_10nov2021.csv',
  '/content/HIST_PAINEL_COVIDBR_2021_Parte2_10nov2021.csv'
]:
  convert_csv_into_table(
      conn,
      cursor,
      path,
      'hist_painel_covidbr_2020_parte1nov2021',
      {
        'regiao': 'regiao',
        'estado': 'estado',
        'municipio': 'municipio',
        'coduf': 'coduf',
        'codmun': 'codmun',
        'codRegiaoSaude': 'codRegiaoSaude',
        'nomeRegiaoSaude': 'nomeRegiaoSaude',
        'data': 'data',
        'semanaEpi': 'semanaEpi',
        'populacaoTCU2019': 'populacaoTCU2019',
        'casosAcumulado': 'casosAcumulado',
        'casosNovos': 'casosNovos',
        'obitosAcumulado': 'obitosAcumulado',
        'obitosNovos': 'obitosNovos',
        'Recuperadosnovos': 'recuperadosNovos',
        'emAcompanhamentoNovos': 'emAcompanhamentoNovos',
        'interior/metropolitana': 'interior_metropolitana'
      },
      {'sep': ';'}
  )

In [ ]:
convert_csv_into_table(
    conn,
    cursor,
    'HIST_PAINEL_COVIDBR_2020_Parte1_10nov2021.csv',
    'hist_painel_covidbr_2020_parte1nov2021',
    {
      'regiao': 'regiao',
      'estado': 'estado',
      'municipio': 'municipio',
      'coduf': 'coduf',
      'codmun': 'codmun',
      'codRegiaoSaude': 'codRegiaoSaude',
      'nomeRegiaoSaude': 'nomeRegiaoSaude',
      'data': 'data',
      'semanaEpi': 'semanaEpi',
      'populacaoTCU2019': 'populacaoTCU2019',
      'casosAcumulado': 'casosAcumulado',
      'casosNovos': 'casosNovos',
      'obitosAcumulado': 'obitosAcumulado',
      'obitosNovos': 'obitosNovos',
      'Recuperadosnovos': 'recuperadosNovos',
      'emAcompanhamentoNovos': 'emAcompanhamentoNovos',
      'interior/metropolitana': 'interior_metropolitana'
    },
    {'sep': ';'}
)

In [ ]:
df = pd.read_csv("..data/external/HIST_PAINEL_COVIDBR_2020_Parte1_10nov2021.csv", sep=';')
df.head()

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana
0,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-25,9,210147125.0,0,0,0,0,NaN,NaN,NaN
1,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-26,9,210147125.0,1,1,0,0,NaN,NaN,NaN
2,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-27,9,210147125.0,1,0,0,0,NaN,NaN,NaN
3,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-28,9,210147125.0,1,0,0,0,NaN,NaN,NaN
4,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-29,9,210147125.0,2,1,0,0,NaN,NaN,NaN


In [ ]:
cursor.execute(
    "CREATE TABLE data_2020_2021 AS " \
    "SELECT regiao, estado, coduf, data, semanaEpi, populacaoTCU2019, casosAcumulado, " \
    "casosNovos, obitosAcumulado, obitosNovos " \
    "from hist_painel_covidbr_2020_parte1nov2021 WHERE municipio IS NOT NULL"
)

In [ ]:
cursor.execute(
    """
    CREATE TABLE analise_obito (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        data VARCHAR(15),
        obitosNovos INTEGER,
        centro_oeste INTEGER,
        nordeste INTEGER,
        norte INTEGER,
        sudeste INTEGER,
        sul INTEGER
    );
    """
)

In [ ]:
convert_csv_into_table(
    conn,
    cursor,
    'analiseobito.csv',
    'analise_obito',
    {
      'Brasil': 'obitosNovos',
      'Centro-Oeste': 'centro_oeste',
      'Nordeste': 'nordeste',
      'Sudeste': 'sudeste',
      'Norte': 'norte',
      'Sul': 'sul',
      'data': 'data'
    }
)

In [ ]:
conn.commit()

In [ ]:
conn.close()